# espy


In [10]:
print("We simulate errors in python!")

We simulate errors in python!


# Noise simulation  
### Creating random test data

In [11]:
import numpy
from espy.channelNoiseSimulator import channelNoiseSimulator

#set seed
numpy.random.seed(100)

cns = channelNoiseSimulator()
print(cns.createRandomBitsString(20))

00111100000100001001


# Noise simulation  
### Simulate simple errors

In [12]:
import numpy
from espy.channelNoiseSimulator import channelNoiseSimulator
#seed
numpy.random.seed(100)
cns = channelNoiseSimulator()

data = cns.createRandomBitsString(20)
noisyData = cns.randomiseBitsString(data,0.1)

print(data)
print(noisyData)
# highlight differences
print(''.join(str(e) for e in cns.compareAndHighlightDifferences(data,noisyData)))

00111100000100001001
00111100000100000000
Differences found: 2
00000000000000001001


# Noise simulation  
### Simulate burst errors

In [13]:
import numpy
from espy.channelNoiseSimulator import channelNoiseSimulator
#seed
numpy.random.seed(100)
cns = channelNoiseSimulator()

data = cns.createRandomBitsString(100)
noisyData = cns.randomiseBitsBurstString(data,0.01,0.9,8)

print(data)
print(noisyData)

# highlight differences
print(''.join(str(e) for e in cns.compareAndHighlightDifferences(data,noisyData)))

0011110000010000100101000111001001111100111111110111001101011100100111000000101010011000111100110010
0011110000010000100101000111001001111100111111110111001101011100100111000111010110011000111100110010
Differences found: 7
0000000000000000000000000000000000000000000000000000000000000000000000000111111100000000000000000000


# Cyclic Redundancy Check

In [14]:
from espy.crc import CyclicRedundancyCheck

crc32 = CyclicRedundancyCheck("1101")

msg = "10100100101"
print(crc32.compute_crc(msg))
msg_with_red = crc32.crc_encode(msg)
print(msg_with_red)

101
10100100101101


In [15]:
print(crc32.crc_check(msg_with_red))

True
